# Data Loading and Processing (pytorch)

In [5]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset

from collections import Counter
from sklearn.externals import joblib


In [8]:
# import images
from skimage import io, transform

In [9]:
import pathlib as plib
import pdb

In [10]:
#Globals
ROOT = plib.Path("/Users/hjsong/Workspace/Luna")
nRuns = 5 #number of test runs per unit
RANDOM_SEED = 15;

In [14]:
cd $ROOT

/Users/hjsong/Workspace/Luna


In [24]:
a = np.array([1,2,3,4,5,6]); print(a.shape)

(6,)


In [25]:
a.reshape(-1,2)

array([[1, 2],
       [3, 4],
       [5, 6]])

In [18]:
ll data/DataMatrix/

total 47160
-rw-r--r--  1 hjsong  staff   146344 Apr 25 11:17 D_0202.pkl
-rw-r--r--  1 hjsong  staff   169912 Apr 19 13:18 D_0202_0316.pkl
-rw-r--r--  1 hjsong  staff   293460 Apr 19 13:18 D_0202_0316_0412.pkl
-rw-r--r--  1 hjsong  staff    28044 Apr 25 11:17 D_0316.pkl
-rw-r--r--  1 hjsong  staff   177322 Apr 25 11:17 D_03_26.pkl
-rw-r--r--  1 hjsong  staff   124064 Apr 19 13:18 D_0412.pkl
-rw-r--r--  1 hjsong  staff  4048249 Apr 25 11:17 D_absRange.pkl
-rw-r--r--  1 hjsong  staff   691159 Apr 19 12:46 D_all_04_19.pkl
-rw-r--r--  1 hjsong  staff   691159 Apr 19 13:18 D_all_comparison.pkl
-rw-r--r--  1 hjsong  staff   356266 Apr 16 10:56 D_extended.pkl
-rw-r--r--  1 hjsong  staff   156264 Apr 16 13:28 D_kgb_cracked.pkl
-rw-r--r--  1 hjsong  staff   154991 Apr 16 13:56 D_kgb_cracked2.pkl
-rw-r--r--  1 hjsong  staff  4048249 Apr 25 11:17 D_rateRange.pkl
-rw-r--r--  1 hjsong  staff  4008571 Apr 26 13:56 D_responseRange_04_26.pkl
-rw-r--r--  1 hjsong  staff   697196 Apr 26 17:36 D_response

In [10]:
Dataset.__init__

Init signature: Dataset()
Docstring:     
An abstract class representing a Dataset.

All other datasets should subclass it. All subclasses should override
``__len__``, that provides the size of the dataset, and ``__getitem__``,
supporting integer indexing in range from 0 to len(self) exclusive.
File:           ~/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/torch/utils/data/dataset.py
Type:           type


In [62]:
class LunaCsvDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        - csv_file (string): path to the csv file
        - root_dir (string): directory with all the images
        - transform (callable, optional): optional transform to be applied to a sample
        """
        all_data = pd.read_csv(csv_file, header=0, index_col=0)
        is_not_fraud = [False if 'Fraud' in all_data.state[i] else True for i in range(len(all_data))]
        all_data = all_data[is_not_fraud]        
        self.data = np.array(all_data.drop('state', axis=1)) 
        self.target = np.array(all_data['state'])
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        "returns a sample of format (feature 1,...,featureD, class)"
        return (self.data[idx, :], self.target[idx])
    

In [ ]:
class LunaPickleDataset(Dataset):
    def __init__(self, pk_file, root_dir, transform=None):
        """
        - pk_file (string): path to the pickled file
        - root_dir (string): directory with all the images
        - transform (callable, optional): optional transform to be applied to a sample
        """
        all_data = pd.read_csv(csv_file, header=None) #todo
        is_not_fraud = [False if 'Fraud' in all_data.state[i] else True for i in range(len(all_data))]
        all_data = all_data[is_not_fraud]
        self.data = np.array(all_data.iloc[:, 1:-1]) #ignore first column (patient ID)
        self.label_dict = {'KGB':0, 'Cracked':1, 'Repaired':2, 'NoPSA':0, }
        self.target = np.array(all_data.state.map(
            ))
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        "returns a sample of format (feature 1,...,featureD, class)"
        return (self.data[idx, :], self.target[idx])
    
    def set_label_dict(self, new_dict):
        self.label_dict = new_dict

In [63]:
fpath = './data/DataMatrix/D_rr_rrr.csv'
d = pd.read_csv(fpath, header=0, index_col=0);
d.columns

Index(['0_rr', '1_rr', '2_rr', '3_rr', '4_rr', '5_rr', '6_rr', '7_rr', '8_rr',
       '9_rr', '10_rr', '11_rr', '12_rr', '13_rr', '14_rr', '15_rr', '16_rr',
       '17_rr', '18_rr', '19_rr', '20_rr', '21_rr', '22_rr', '23_rr', '24_rr',
       '25_rr', '26_rr', '27_rr', '28_rr', '29_rr', '30_rr', '31_rr', '32_rr',
       '33_rr', '34_rr', '35_rr', '36_rr', '37_rr', '38_rr', '39_rr', '40_rr',
       '41_rr', '42_rr', '43_rr', '44_rr', '45_rr', '46_rr', '47_rr', '0_rrr',
       '1_rrr', '2_rrr', '3_rrr', '4_rrr', '5_rrr', '6_rrr', '7_rrr', '8_rrr',
       '9_rrr', '10_rrr', '11_rrr', '12_rrr', '13_rrr', '14_rrr', '15_rrr',
       '16_rrr', '17_rrr', '18_rrr', '19_rrr', '20_rrr', '21_rrr', '22_rrr',
       '23_rrr', '24_rrr', '25_rrr', '26_rrr', '27_rrr', '28_rrr', '29_rrr',
       '30_rrr', '31_rrr', '32_rrr', '33_rrr', '34_rrr', '35_rrr', '36_rrr',
       '37_rrr', '38_rrr', '39_rrr', '40_rrr', '41_rrr', '42_rrr', '43_rrr',
       '44_rrr', '45_rrr', '46_rrr', '47_rrr', 'sn', 'state'],
 

In [64]:
dataset = LunaCsvDataset(fpath,root_dir='./data')

In [65]:
dataset.data.shape

(673, 97)

In [66]:
dataset.target

array(['Cracked', 'Cracked', 'Cracked', 'Cracked', 'Cracked', 'Cracked',
       'Cracked', 'Cracked', 'Cracked', 'Cracked', 'Cracked', 'Cracked',
       'Cracked', 'Repaired', 'Repaired', 'Repaired', 'Repaired',
       'Repaired', 'Repaired', 'Repaired', 'Repaired', 'Repaired',
       'Repaired', 'Repaired', 'Repaired', 'Repaired', 'Repaired',
       'Repaired', 'Repaired', 'Repaired', 'Repaired', 'Repaired',
       'Repaired', 'Repaired', 'Repaired', 'Repaired', 'Repaired',
       'Repaired', 'Repaired', 'Repaired', 'Repaired', 'Repaired',
       'Repaired', 'Repaired', 'Repaired', 'Repaired', 'Repaired',
       'Repaired', 'Repaired', 'Repaired', 'Repaired', 'Repaired',
       'Repaired', 'Repaired', 'Repaired', 'Repaired', 'Repaired',
       'Repaired', 'Repaired', 'Repaired', 'Repaired', 'Repaired',
       'Repaired', 'Repaired', 'Repaired', 'Repaired', 'Repaired',
       'Repaired', 'Repaired', 'Repaired', 'Repaired', 'Repaired',
       'Repaired', 'Repaired', 'Repaired', 'Repaire

In [50]:
d[~flag]

TypeError: bad operand type for unary ~: 'list'

In [87]:
dataset = LunaCsvDataset(fpath, './data')
a = dataset[10]

In [88]:
a

(array([1, 1, 1, 1, 1, '1', 3, 1, 1], dtype=object), 0)

In [84]:
type(dataset.data.iloc[10,:])

pandas.core.series.Series

In [89]:
dataset[40]

In [99]:
def count_files(dirpath, fmt=None):
    """
    Args
    - dirpath (Path object): directory to inspect
    - fmt (string): file format including '.', eg: '.png', '.jpg', '.csv'
    Returns a dictionary of counts for each filetype in dirpath"""
    import collections
    counts =  collections.Counter(f.suffix for f in dirpath.iterdir())
    if fmt is None:
        return counts
    else:
        return counts[fmt]

In [ ]:
class DogCatDataset(Dataset):
    def __init__(self, dirpath, animalType):
        """
        Args
        - dirpath (Path object): path to a folder that contains 
        'dogs' and 'cats' subfolers. For instance, `data/dogscats/train` or 
        `data/dogscats/test`
        - animalType (string): 'dog' or 'cat'
        
        Sample of this dataset will be a dictionary of {"image": img, "class": 0 or 1}
        """
        self.animalType = animalType
        self.animalClass = 0 if animalType == 'dog' else 1
        self.full_dirpath = dirpath / (animalType + "s")
        self.num_data = count_files(self.full_dirpath, fmt='.jpg')
    def __len__(self):
        return self.num_data
    def __getitem__(self, idx):
        img_name = self.full_dirpath / f'{self.animalType}.{idx}.jpg'
        
    
        
        